In [ ]:
!python3 model_inspect.py \
    --runmode saved_model \
    --model_name efficientdet-d5 \
    --ckpt_path /Users/arif/Downloads/efficientdet-d5 \
    --saved_model_dir "/Users/arif/Documents/Pet Feeder/Software/AI/cat_feeder_project/od_tf"

In [ ]:
!python3 create_onnx.py \
    --input_size 1280,1280 \
    --saved_model "/Users/arif/Documents/Pet Feeder/Software/AI/cat_feeder_project/od_tf" \
    --onnx "/Users/arif/Documents/Pet Feeder/Software/AI/cat_feeder_project/effdet_d5.onnx" \
    --input_format NCHW \
    --preprocessor scale_range

# Logging config 1-time only

In [4]:
import logging


class CurrentModuleFilter(logging.Filter):
    def __init__(self, module_name):
        super().__init__()
        self.module_name = module_name

    def filter(self, record):
        # Only allow log messages from the specified module
        return record.name == self.module_name


# Create a logger with a specific name (usually the name of the module)
logger_name = "onnx"  # Replace with your module's name
logger = logging.getLogger(logger_name)
logger.setLevel(logging.INFO)
logger.propagate = False

# Create a logging handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create and set the custom filter
module_filter = CurrentModuleFilter(logger_name)
console_handler.addFilter(module_filter)

log_format = "%(lineno)d : %(message)s"
formatter = logging.Formatter(log_format)
console_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(console_handler)

# Test the logger
logger.debug(f"This message is from the {logger_name} module.")

In [2]:
import onnx
import onnxruntime as ort
import numpy as np
import cv2
from joblib import Parallel, delayed

from easyfsl.samplers import TaskSampler
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision.datasets import OxfordIIITPet

hw.cpufamily: 458787763 , size = 4
The device support i8sdot:1, support fp16:1, support i8mm: 0


# Data

In [3]:
train_set = OxfordIIITPet(
    root="/Users/arif/Downloads/data",
    split="trainval",
    target_types="category",
    download=False,
)

test_set = OxfordIIITPet(
    root="/Users/arif/Downloads/data",
    split="test",
    target_types="category",
    download=False,
)

# ONNX opti

In [5]:
# !onnxsim /Users/arif/Downloads/edgenext_base_usi_in1k_support.onnx /Users/arif/Downloads/edgenext_base_usi_in1k_support_opt.onnx 5 --enable-onnxruntime-optimization
# !onnxsim /Users/arif/Downloads/edgenext_base_usi_in1k_query.onnx /Users/arif/Downloads/edgenext_base_usi_in1k_query_opt.onnx 5 --enable-onnxruntime-optimization
!onnxsim ./test.onnx ./test_opt.onnx

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                   ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add               │ 72             │ 32               │
│ ArgMin            │ 1              │ 1                │
│ Cast              │ 96             │ 1                │
│ Ceil              │ 20             │ 0                │
│ Clip              │ 20             │ 0                │
│ Concat            │ 31             │ 1                │
│ Constant          │ 569            │ 239              │
│ ConstantOfShape   │ 10             │ 0                │
│ Conv              │ 230            │ 230              │
│ Div               │ 36        

# Model Init

In [4]:
test_model = onnx.load("./test_opt.onnx")
onnx.checker.check_model(test_model, full_check=True)

In [5]:
test_ort_sess = ort.InferenceSession("./test_opt.onnx")

# Torch Compose

In [13]:
import torch
import timm
from PIL import Image
from torchvision.transforms import v2, InterpolationMode

image = Image.open("/Users/arif/Downloads/beignets-task-guide.png").convert("RGB")

transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.uint8, scale=True),
        v2.Resize([292, 292], interpolation=InterpolationMode.BICUBIC, antialias=None),
        v2.CenterCrop(260),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

image = transforms(image)
logger.info(f"image {image.dtype, image.shape, image.mean(), image.std()}")

# model = timm.create_model(
#     'timm/tf_efficientnet_b2.ns_jft_in1k',
#     # 'edgenext_base.usi_in1k',
#     pretrained=True,
#     num_classes=0,
#     exportable=True
# )
# model = model.eval()
# torch_outputs = model(transforms(image).unsqueeze(0)).detach().numpy()
# logger.info(f"torch_output {torch_outputs.dtype, torch_outputs.shape, torch_outputs.mean(), torch_outputs.std(), torch_outputs.max(), torch_outputs.min()}")

18 : image (torch.float32, torch.Size([3, 260, 260]), tensor(1.1152), tensor(0.9671))


# CV2 Compose

In [18]:
def preprocess_image(image):
    mean = np.asarray([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.asarray([0.229, 0.224, 0.225], dtype=np.float32)

    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1152, 1152), interpolation=cv2.INTER_CUBIC)

    # w, h = 260, 260
    # y, x = (image.shape[0] - h) // 2, (image.shape[1] - w) // 2
    # image = image[y:y + h, x:x + w]

    image = image.astype(np.float32)
    # image = image.astype(np.float32) / 255.0
    # image = (image - mean) / std
    image = image.transpose([2, 0, 1])

    image = np.expand_dims(image, 0)
    return image


# image = cv2.imread("/Users/arif/Downloads/beignets-task-guide.png", cv2.IMREAD_COLOR)
# image = cv2.imread("/Users/arif/Downloads/149096135_38da34403d_z.jpg", cv2.IMREAD_COLOR)
image = cv2.imread(
    "/Users/arif/Downloads/2126259110_79df1cde52_z.jpg", cv2.IMREAD_COLOR
)
# image = cv2.imread("/Users/arif/Downloads/148363573_db510f0e0a_z.jpg", cv2.IMREAD_COLOR)
image = cv2.imread("CatDataset/support/jico/lp_image.jpeg", cv2.IMREAD_COLOR)
image_proc = preprocess_image(image)
logger.info(
    f"image {image_proc.dtype, image_proc.shape, image_proc.mean(), image_proc.std()}"
)

# support_ort_sess = ort.InferenceSession("./yolov6n_opt.onnx")
support_ort_sess = ort.InferenceSession("/Users/arif/Downloads/yolov6n.onnx")
# outputTensors = ['det_boxes']
# output_names=['num_det', 'det_boxes', 'det_scores', 'det_classes']
# outputTensors = ['x0', 'y0', 'x1', 'y1', 'det_score']
# try:
#     cv2_outputs = support_ort_sess.run(None, {'x.1': image})
# except:
all = support_ort_sess.run(None, {"images": image_proc})
# x0, y0, x1, y1, det_scores = support_ort_sess.run(None, {'images': image_proc})
# x0, y0, x1, y1 = [int(coord) for coord in (x0, y0, x1, y1)]
print(all)
# print(x0, y0, x1, y1, det_scores)

# Define the color of the box and its thickness
color = (0, 255, 0)  # Green
thickness = 2  # Thickness of the lines used to draw the box

# Draw the rectangle on the image
image = cv2.imread(
    "/Users/arif/Downloads/2126259110_79df1cde52_z.jpg", cv2.IMREAD_COLOR
)
# cv2.rectangle(image, (x0[0], y0[0]), (x1[0], y1[0]), color, thickness)

# Display the image
# cv2.imshow('Image with Bounding Box', image)

# logger.info(f"cv2_output {cv2_outputs.dtype, cv2_outputs.shape, cv2_outputs.mean(), cv2_outputs.std(), cv2_outputs.max(), cv2_outputs.min()}")

26 : image (dtype('float32'), (1, 3, 1152, 1152), 84.842224, 46.15985)


[array([   9,   33, 1548, 1545], dtype=int32)]


In [17]:
all

[array([  4,  13, 638, 637], dtype=int32)]

In [30]:
image_cv = cv2.imread("CatDataset/support/jico/jico_test.jpeg", cv2.IMREAD_UNCHANGED)

In [31]:
import MNN.nn as nn
import MNN.cv as cv
import MNN.numpy as np
import numpy as np2
import MNN.expr as expr

print(image_cv.dtype, image_cv.mean(), image_cv.std(), image_cv.max(), image_cv.min())
image = np.asarray(image_cv.tolist(), dtype=np.uint8)
print(image.dtype, image.mean(dtype=np.float64), image.std(), image.max(), image.min())

uint8 123.24284443344645 55.43454773061126 233 0
dtype.uint8 123.06302642822266 55.38330078125 233 0


In [32]:
image_cv.shape

(884, 1920, 3)

In [35]:
image = expr.const(image_cv.tolist(), [884, 1920, 3], expr.NHWC, expr.uint8)
print(image.dtype, image.mean(dtype=np.float64), image.std(), image.max(), image.min())

dtype.uint8 123.06302642822266 55.38330078125 233 0


In [36]:
mean = np.asarray([0.485, 0.456, 0.406], dtype=np.float32)
std = np.asarray([0.229, 0.224, 0.225], dtype=np.float32)

image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
# image = image[:, :, ::-1]
# image = image[:,[2,1,0],...]
print("image cvt", image.dtype, image.shape, image.mean(), image.std())
# image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
# image = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
image = cv.resize(image, (640, 640), interpolation=cv.INTER_CUBIC)
print("image rsz", image.dtype, image.shape, image.mean(), image.std())

# w, h = 640, 640
# y, x = (image.shape[0] - h) // 2, (image.shape[1] - w) // 2
# image = image[y:y + h, x:x + w]

image = image.astype(np.float32) / 255.0
print("image 255", image.dtype, image.shape, image.mean(), image.std())
image = (image - mean) / std
print("image norm", image.dtype, image.shape, image.mean(), image.std())
# image = image.transpose([2, 0, 1]) #! only for onnx

image = np.expand_dims(image, 0)

image cvt dtype.uint8 [884, 1920, 3] 123.06107330322266 55.382450103759766
image rsz dtype.uint8 [640, 640, 3] 123.24080657958984 55.43525314331055
image 255 dtype.float [640, 640, 3] 0.4832761883735657 0.2172865867614746
image norm dtype.float [640, 640, 3] 0.15166887640953064 0.948476254940033


In [152]:
x0, y0, x1, y1 = [int(coord) for coord in (x0, y0, x1, y1)]

# Define the color of the box and its thickness
color = (0, 255, 0)  # Green
thickness = 2  # Thickness of the lines used to draw the box

# Draw the rectangle on the image
image = cv2.imread(
    "/Users/arif/Downloads/2126259110_79df1cde52_z.jpg", cv2.IMREAD_COLOR
)
cv2.rectangle(image, (x0, y0), (x1, y1), color, thickness)

# Display the image
cv2.imshow("Image with Bounding Box", image)

In [125]:
widths = det_boxes[..., 2] - det_boxes[..., 0]
heights = det_boxes[..., 3] - det_boxes[..., 1]

In [136]:
torch.argmax(torch.Tensor(widths) * torch.Tensor(heights))

tensor(1)

In [142]:
import torch

torch.cat([torch.tensor(num_det), torch.tensor(det_boxes), torch.tensor(det_scores)])

RuntimeError: Tensors must have same number of dimensions: got 1 and 2

In [7]:
# import matplotlib.pyplot as plt

# plt.imshow(image.transpose(1, 2, 0))
# plt.axis('off')
# plt.show()

In [8]:
# rtol = 0.1  # Relative tolerance
# atol = 0.1  # Absolute tolerance

# # Compare arrays
# are_close = np.allclose(cv2_outputs, torch_outputs, rtol=rtol, atol=atol)
# print(are_close)

# # Element-wise comparison
# close_elements = np.isclose(cv2_outputs, torch_outputs, rtol=rtol, atol=atol)
# print(close_elements)

# Old model

In [9]:
def calculate_mean_for_label(z_support, support_labels, label):
    return np.mean(z_support[support_labels[support_labels == label]], axis=0)


def fsl_model(support_net, query_net, support_images, support_labels, query_images):
    logger.info(
        f"support_images {support_images.shape, support_images.mean(), support_images.std()}"
    )
    logger.info(
        f"query_images {query_images.shape, query_images.mean(), query_images.std()}"
    )

    # try:
    z_support = support_net.run(None, {"input.1": support_images})[0]
    z_query = query_net.run(None, {"input.1": query_images})[0]
    # except:
    #     z_support = support_net.run(["efficientnet-b2/model/head/dropout/Identity"], {'images': support_images})[0]
    #     z_query = query_net.run(["efficientnet-b2/model/head/dropout/Identity"], {'images': query_images})[0]

    logger.info(f"z_support {z_support.shape, z_support.mean(), z_support.std()}")
    logger.info(f"z_query {z_query.shape, z_query.mean(), z_query.std()}")

    n_way = len(np.unique(support_labels))
    logger.debug(f"support_labels {support_labels}")
    logger.info(f"n_way {n_way}")

    z_proto = Parallel(n_jobs=1)(
        delayed(calculate_mean_for_label)(z_support, support_labels, label)
        for label in range(n_way)
    )
    z_proto = np.stack(z_proto)

    logger.info(f"z_proto {z_proto.shape, z_proto.mean(), z_proto.std()}")
    logger.debug(f"z_proto {z_proto}")

    # Compute the euclidean distance from queries to prototypes
    # dists = cdist(z_query, z_proto)
    dists = np.linalg.norm(z_query[:, np.newaxis] - z_proto, axis=2)
    logger.info(f"dists {dists.shape, dists.mean(), dists.std()}")
    logger.debug(f"dists {dists}")

    # And here is the super complicated operation to transform those distances into classification scores!
    scores = -dists
    logger.info(f"scores {scores.shape, scores.mean(), scores.std()}")
    logger.debug(f"scores {scores}")
    return scores

In [6]:
N_WAY = 4  # Number of classes in a task
N_SHOT = 5  # Number of images per class in the support set
N_QUERY = 30  # Number of images per class in the query set
N_EVALUATION_TASKS = 100

# The sampler needs a dataset with a "get_labels" method. Check the code if you have any doubt!
test_set.get_labels = lambda: test_set._labels
test_sampler = TaskSampler(
    test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=0,
    pin_memory=False,
    collate_fn=test_sampler.episodic_collate_fn,
)

(
    example_support_images,
    example_support_labels,
    example_query_images,
    example_query_labels,
    example_class_ids,
) = next(iter(test_loader))

logger.info(
    f"{len(example_support_images), len(example_support_labels), len(example_query_images), len(example_query_labels), len(example_class_ids)}"
)

In [8]:
def evaluate_on_one_task(
    support_images, support_labels, query_images, query_labels, n_way
):
    # Get model predictions
    # predictions = np.argmax(fsl_model(support_ort_sess, query_ort_sess, support_images, support_labels, query_images), 1)
    predictions = test_ort_sess.run(
        None,
        {"x.1": support_images, "onnx::Cast_1": support_labels, "x.149": query_images},
    )[0]
    logger.info(f"predictions {predictions}")
    logger.info(f"query_labels {query_labels}")
    # query_labels = torch.from_numpy(query_labels)
    accuracy = np.sum(predictions == query_labels) / len(query_labels)

    return accuracy * 100


def evaluate(data_loader):
    for episode_index, (
        support_images,
        support_labels,
        query_images,
        query_labels,
        class_ids,
    ) in tqdm(enumerate(data_loader), total=len(data_loader)):
        accuracy = evaluate_on_one_task(
            support_images, support_labels, query_images, query_labels, len(class_ids)
        )
        # break

    print(f"Model tested on {len(data_loader)} tasks.")
    print(f"Accuracy: {accuracy:.2f}%")


evaluate(test_loader)

100%|██████████| 100/100 [08:20<00:00,  5.01s/it]

Model tested on 100 tasks.
Accuracy: 100.00%


# Export TF

In [16]:
!python3 /Users/arif/Documents/tpu/models/official/efficientnet/export_model.py --model_name=efficientnet-b2 --ckpt_dir=/Users/arif/Downloads/noisy_student_efficientnet-b2 --output_saved_model_dir="./FSL Models/test_1" --output_tflite="./FSL Models/test_1.tflite" --endpoint_name "global_pool" --image_size 260 --batch_size 1 --quantize=False

1
I0129 20:31:38.170017 8057855040 efficientnet_builder.py:225] global_params= GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.3, data_format='channels_last', num_classes=1000, width_coefficient=1.1, depth_coefficient=1.2, depth_divisor=8, min_depth=None, survival_prob=0.8, relu_fn=<function swish at 0x11e4c8ee0>, batch_norm=<function eval_batch_norm at 0x15f6314c0>, use_se=True, se_coefficient=None, local_pooling=None, condconv_num_experts=None, clip_projection_output=False, blocks_args=['r1_k3_s11_e1_i32_o16_se0.25', 'r2_k3_s22_e6_i16_o24_se0.25', 'r2_k5_s22_e6_i24_o40_se0.25', 'r3_k3_s22_e6_i40_o80_se0.25', 'r3_k5_s11_e6_i80_o112_se0.25', 'r4_k5_s22_e6_i112_o192_se0.25', 'r1_k3_s11_e6_i192_o320_se0.25'], fix_head_stem=None, use_bfloat16=None)
I0129 20:31:38.173455 8057855040 efficientnet_model.py:121] round_filter input=32 output=32
I0129 20:31:38.176726 8057855040 efficientnet_model.py:121] round_filter input=32 output=32
I0129 20:31:38.176765 805785

# support and query TF

In [ ]:
!python3 /Users/arif/Documents/tpu/models/official/efficientnet/export_model.py --model_name=efficientnet-b2 --ckpt_dir=/Users/arif/Downloads/noisy_student_efficientnet-b2 --output_saved_model_dir="./FSL Models/support_20" --output_tflite="./FSL Models/support_20.tflite" --endpoint_name "global_pool" --image_size 260 --batch_size 20 --quantize=False -v 0
!python3 /Users/arif/Documents/tpu/models/official/efficientnet/export_model.py --model_name=efficientnet-b2 --ckpt_dir=/Users/arif/Downloads/noisy_student_efficientnet-b2 --output_saved_model_dir="./FSL Models/support_20" --output_tflite="./FSL Models/support_20.tflite" --endpoint_name "global_pool" --image_size 260 --batch_size 20 --quantize=False -v 0

# TFlite to ONNX

In [12]:
!python3 -m tf2onnx.convert --opset 18 --tflite "./FSL Models/test_1.tflite" --output "./FSL Models/test_1_tflite.onnx" --inputs-as-nchw images --verbose

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-01-29 19:35:25,908 - WARNING - tf2onnx: ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-01-29 19:35:25,908 - INFO - tf2onnx: inputs: None
2024-01-29 19:35:25,908 - INFO - tf2onnx: outputs: None
2024-01-29 19:35:25,909 - INFO - tf2onnx.tfonnx: Using tensorflow=2.15.0, onnx=1.15.0, tf2onnx=1.16.1/15c810
2024-01-29 19:35:25,909 - INFO - tf2onnx.tfonnx: Using opset <onnx, 18>
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-29 19:35:26,029 - VERBOSE - tf2onnx.tfonnx: Mapping TF node to ONNX node(s)
2024-01-29 19:35:26,161 - VERBOSE - tf2onnx.tfonnx: Mapping TF node to ONNX node(s)
2024-01-29 19:35:26,316 

In [9]:
!python3 -m tf2onnx.convert --opset 18 --tflite "./FSL Models/support_20.tflite" --output "./FSL Models/support_20_tflite.onnx" --inputs-as-nchw images
!python3 -m tf2onnx.convert --opset 18 --tflite "./FSL Models/query_160.tflite" --output "./FSL Models/query_160_tflite.onnx" --inputs-as-nchw images

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-01-29 19:02:03,678 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-01-29 19:02:03,683 - INFO - Using tensorflow=2.15.0, onnx=1.15.0, tf2onnx=1.16.1/15c810
2024-01-29 19:02:03,684 - INFO - Using opset <onnx, 18>
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-29 19:02:04,091 - INFO - Optimizing ONNX model
2024-01-29 19:02:07,357 - INFO - After optimization: Cast -24 (24->0), Const -5 (254->249), Identity -1 (1->0), Reshape +23 (23->46), Transpose -460 (461->1)
2024-01-29 19:02:07,404 - INFO - 
2024-01-29 19:02:07,404 - INFO - Successfully converted TensorFlow model ./FSL Models/support_20.

# TF to ONNX

In [1]:
!python3 -m tf2.inspector --mode=export --model_name=efficientdet-d5 \
    --model_dir="/Users/arif/Downloads/efficientdet-d5" \
    --image_size=1280 \
    --saved_model_dir="/Users/arif/Documents/Pet_Feeder/Software/AI/cat_feeder_project/od_tf" \
    --hparams="nms_configs.method='hard', nms_configs.iou_thresh=0.5, nms_configs.sigma=0.0, mean_rgb=0.0, stddev_rgb=1.0, scale_range=True"

/Users/arif/anaconda3/envs/mnn/bin/python3: Error while finding module specification for 'tf2.inspector' (ModuleNotFoundError: No module named 'tf2')


In [2]:
!python3 -m tf2onnx.convert -h

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: convert.py [-h] [--input INPUT] [--graphdef GRAPHDEF]
                  [--saved-model SAVED_MODEL] [--tag TAG]
                  [--signature_def SIGNATURE_DEF]
                  [--concrete_function CONCRETE_FUNCTION]
                  [--checkpoint CHECKPOINT] [--keras KERAS] [--tflite TFLITE]
                  [--tfjs TFJS] [--large_model] [--output OUTPUT]
                  [--inputs INPUTS] [--outputs OUTPUTS]
                  [--ignore_default IGNORE_DEFAULT]
                  [--use_default USE_DEFAULT] [--rename-inputs RENAME_INPUTS]
                  [--rename-outputs RENAME_OUTPUTS] [--use-graph-names]
                  [--opset OPSET] [--dequantize] [--custom-ops CUSTOM_OPS]
                  [--extra_opse

!python3 -m tf2onnx.convert --saved-model="/Users/arif/Documents/Pet_Feeder/Software/AI/cat_feeder_project/od_tf" \
    --output="/Users/arif/Documents/Pet_Feeder/Software/AI/cat_feeder_project/od_tf.onnx" \
    --opset=18 \
    --inputs-as-nchw images \
    --signature_def serving_default \
    --tag serve

In [18]:
!python3 -m tf2onnx.convert --saved-model="/Users/arif/Documents/Pet_Feeder/Software/AI/cat_feeder_project/od_tf" \
    --output="/Users/arif/Documents/Pet_Feeder/Software/AI/cat_feeder_project/od_tf.onnx" \
    --opset=17 \
    --inputs-as-nchw images \
    --signature_def serving_default \
    --tag serve

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-02-19 16:14:18,927 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-02-19 16:14:30,651 - WARNING - Importing a function (__inference_predict_71870) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2024-02-19 16:14:35,395 - WARNING - Importing a function (__inference_efficientnet-b5_layer_call_and_return_conditional_losses_154075) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2024-02-19 16:14:36,275 - WARNING - Importing a function (__inference_blocks_36_layer_call_and_return_conditional_losses_181183) with ops with unsaved custom g

In [1]:
!python3 -m tf2onnx.convert --opset 17 --saved-model "/Users/arif/Documents/Pet Feeder/Software/AI/cat_feeder_project/od_tf" --output od_tf.onnx --inputs-as-nchw input

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-02-18 19:48:47,178 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-02-18 19:48:47,184 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-02-18 19:49:38,915 - INFO - Fingerprint not found. Saved model loading will continue.
2024-02-18 19:49:38,915 - INFO - path_and_singleprint metric could not be logged. Saved model loading will continue.
2024-02-18 19:49:39,039 - INFO - Signatures found in model: [serving_default].
2024-02-18 19:49:39,039 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-02-18 19:49:39,042 - INFO - Output names: ['detections:0

In [18]:
!python3 -m tf2onnx.convert --opset 18 --saved-model "./FSL Models/test_1" --output test_1_tf.onnx --inputs-as-nchw input --verbose

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-01-29 20:33:51,395 - WARNING - tf2onnx: ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-01-29 20:33:51,396 - WARNING - tf2onnx.tf_loader: '--tag' not specified for saved_model. Using --tag serve
2024-01-29 20:33:56,101 - INFO - absl: Fingerprint not found. Saved model loading will continue.
2024-01-29 20:33:56,102 - INFO - absl: path_and_singleprint metric could not be logged. Saved model loading will continue.
2024-01-29 20:33:56,111 - INFO - tf2onnx.tf_loader: Signatures found in model: [serving_default].
2024-01-29 20:33:56,111 - WARNING - tf2onnx.tf_loader: '--signature_def' not specified, using first signature:

In [7]:
!python3 -m tf2onnx.convert --opset 18 --saved-model "./FSL Models/support_20" --output support_20_tf.onnx --inputs-as-nchw input
!python3 -m tf2onnx.convert --opset 18 --saved-model "./FSL Models/query_160" --output query_160_tf.onnx --inputs-as-nchw input

/Users/arif/anaconda3/envs/mnn/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-01-29 18:46:51,843 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-01-29 18:46:51,844 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-01-29 18:46:56,438 - INFO - Fingerprint not found. Saved model loading will continue.
2024-01-29 18:46:56,439 - INFO - path_and_singleprint metric could not be logged. Saved model loading will continue.
2024-01-29 18:46:56,447 - INFO - Signatures found in model: [serving_default].
2024-01-29 18:46:56,447 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-01-29 18:46:56,448 - INFO - Output names: ['output']
Typ